# Proyek Prediksi Risiko Diabetes (Advanced Extraction)

Notebook ini diperbarui khusus untuk menangani struktur file UCI yang kompleks (Nested Archive). Kode ini akan:
1. Mengekstrak file `.zip`.
2. Otomatis mendeteksi dan mengekstrak file `.tar.Z` di dalamnya.
3. Memfilter file dokumentasi (`Index`, `README`) untuk menemukan data yang benar.

**Langkah Awal:**
Upload file `diabetes.zip` Anda.

## 1. Import Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

# Library handling file & system
from google.colab import files
import io
import zipfile
import os

## 2. Upload dan Ekstrak Bertingkat (.zip -> .tar.Z)
Kode berikut menggunakan perintah sistem `tar` untuk menangani file `.tar.Z` yang sulit dibuka oleh library Python standar.

In [ ]:
print("Silakan upload file 'diabetes.zip':")
uploaded = files.upload()

zip_filename = next(iter(uploaded))
print(f"\n[1] Mengekstrak ZIP: {zip_filename}...")

# 1. Ekstrak ZIP
with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall()

# 2. Cari dan Ekstrak file .tar.Z (jika ada)
for root, dirs, files_in_dir in os.walk('.'):
    for filename in files_in_dir:
        if filename.endswith('.tar.Z') or filename.endswith('.tar'):
            full_path = os.path.join(root, filename)
            print(f"[2] Ditemukan arsip bersarang: {filename}")
            print("    Mengekstrak dengan 'tar'...")
            # Menggunakan command line system untuk ekstrak (lebih kuat untuk format lama)
            os.system(f"tar -xf {full_path}")

# 3. Cari File Data yang Sebenarnya
print("\n[3] Mencari file data...")
data_file_name = None

for root, dirs, files_in_dir in os.walk('.'):
    for filename in files_in_dir:
        # Abaikan file sistem, notebook sendiri, atau file arsip
        if filename.startswith('.') or filename == zip_filename or filename.endswith('.ipynb'):
            continue
        if filename.endswith('.zip') or filename.endswith('.tar.Z') or filename.endswith('.tar'):
            continue
            
        # Abaikan dokumentasi (Index, README, dst)
        lower_name = filename.lower()
        if 'index' in lower_name or 'readme' in lower_name:
            print(f"    -> Mengabaikan dokumentasi: {filename}")
            continue
            
        # Jika lolos filter, ini kemungkinan besar file datanya
        # (Biasanya bernama 'diabetes-data' atau 'data')
        data_file_name = os.path.join(root, filename)
        break

if data_file_name:
    print(f"\n[SUKSES] Target data ditemukan: {data_file_name}")
    
    nama_kolom = [
        'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
        'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
    ]

    # Membaca data (biasanya delimiter tab atau spasi untuk format .tar.Z, kita coba deteksi otomatis)
    # engine='python' lebih fleksibel menangani separator
    try:
        diabetes_dataset = pd.read_csv(data_file_name, header=None, names=nama_kolom, sep=None, engine='python')
    except:
        # Fallback jika gagal, coba separator koma standar
        diabetes_dataset = pd.read_csv(data_file_name, header=None, names=nama_kolom)
    
    display(diabetes_dataset.head())
else:
    print("\n[ERROR] Tidak dapat menemukan file data yang valid setelah ekstraksi.")

## 3. Analisis Data Singkat

In [ ]:
if data_file_name:
    print(f"Ukuran data: {diabetes_dataset.shape}")
    display(diabetes_dataset.describe())
    
    print("\nJumlah data per label:")
    print(diabetes_dataset['Outcome'].value_counts())

## 4. Preprocessing Data (Standarisasi)

In [ ]:
if data_file_name:
    X = diabetes_dataset.drop(columns='Outcome', axis=1)
    Y = diabetes_dataset['Outcome']

    scaler = StandardScaler()
    scaler.fit(X)
    standarized_data = scaler.transform(X)

    X = standarized_data
    print("Data berhasil distandarisasi.")

## 5. Train/Test Split & Modeling

In [ ]:
if data_file_name:
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)

    classifier = svm.SVC(kernel='linear')
    classifier.fit(X_train, Y_train)

    train_acc = accuracy_score(classifier.predict(X_train), Y_train)
    test_acc = accuracy_score(classifier.predict(X_test), Y_test)

    print(f'Akurasi Training: {train_acc * 100:.2f}%')
    print(f'Akurasi Testing : {test_acc * 100:.2f}%')

## 6. Prediksi Data Baru

In [ ]:
if data_file_name:
    # Contoh input manual
    input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

    input_as_numpy = np.asarray(input_data).reshape(1, -1)
    std_data = scaler.transform(input_as_numpy)
    prediction = classifier.predict(std_data)

    print(f"Prediksi: {'BERISIKO Diabetes' if prediction[0] == 1 else 'TIDAK berisiko'}")